# Bulk Delete Manual Sharing to All Internal Users

As a system admin, it can happen that you find yourself looking at a Salesforce org in which lots of records have been manually shared to a group called "AllInternalUsers" (see <a href="https://success.salesforce.com/answers?id=9063A000000iW4zQAE" target="_blank">here</a> and <a href="https://success.salesforce.com/answers?id=90630000000i1AWAAY" target="_blank">here</a>). For data protection reasons or to improve user experience, you might find yourself wanting to delete *all* such rules and implement a more systematic sharing schema. This can be frustrating because Salesforce provides no native way of doing this in the point-and-click system admin settings.

This example shows a simple process for managing this with a short Python script ...

In [1]:
from simple_salesforce import Salesforce
import json
import pandas as pd

Load credentials from a locally stored JSON file in the format:

<code>{"username": "me@gmail.com",
 "organizationId": "00Db0000000dhJlJLNJ",
 "password": "password123",
 "security_token": "ABCSKNFOQNlknksndkmsdKASNDKNDkn"}
</code>

In [2]:
credentials = json.load(open("./credentials/salesforce_credentials.json"))

Create Salesforce API connection:

In [3]:
sf = Salesforce(password = credentials["password"],
                username = credentials["username"],
                organizationId = credentials["organizationId"],
                security_token = credentials["security_token"])

In this example I will query for manual shares on the Account object, hence I am querying the object called <code>AccountShare</code>.

The <code>Share</code> object exists for all objects, so to get shares on a different object just add "Share" to the target object name.

In [4]:
query = """
SELECT 
Id, AccountId, UserOrGroupId
FROM AccountShare
"""
result = sf.query_all(query)
a_s = pd.DataFrame(result['records'])
a_s.drop("attributes", axis = 1, inplace = True)
a_s.tail()

,Id,AccountId,UserOrGroupId
27131,00rb000009QLQH9AAP,001b000003wOHuJAAW,00Gb0000002Cb2rEAC
27132,00rb000009ypYeiAAE,001b000003wPNXXAA4,00Gb0000002Cb2rEAC
27133,00rb000009hkmp7AAA,001b000003wQE8kAAG,00Gb0000002Cb2rEAC
27134,00r0N0000ATH8goQQD,001b000003zn1N2AAI,00Gb0000002Cb2rEAC
27135,00rb000009s1bOvAAI,001b000003znAN9AAM,00Gb0000002Cb2rEAC


We need the API name for the <code>AllInternalUser</code> group, which can be obtained with the following code:

In [5]:
query = """
SELECT 
Id, DeveloperName
FROM Group
WHERE DeveloperName='AllInternalUsers'
"""
result = sf.query_all(query)
AIU_id = result["records"][0]["Id"]
AIU_id

'00Gb0000001JX1ZEAW'

We can now filter the DataFrame to find all shares to this group, to get the list of records to delete:

In [6]:
shares_to_delete = list(a_s[a_s["UserOrGroupId"].str.contains(AIU_id)]["Id"])
shares_to_delete[:5]

['00r0N0000AKmlq1QQB',
 '00r0N0000AQ3aVTQQZ',
 '00r0N0000AQ5GFKQQ3',
 '00r0N0000AQ6ygUQQR',
 '00r0N0000AQ7PHvQQN']

This is the list of IDs for the <code>AccountShares</code> to the <code>AllInternalUser</code> *not* the Accounts themselves.

So we can now iterate through the list deleting records from Salesforce:

In [ ]:
# Not going to run this as I don't need to in my org!
if len(shares_to_delete)>0:
    print("{} sharing records to delete".format(len(shares_to_delete)))
    
    count=0

    for share_id in shares_to_delete:

        try:
            sf.AccountShare.delete(record_id=share_id, raw_response=True)
            count+=1
        except:
            pass

    print("{} sharing records successfully deleted.".format(count))

else:
    print("No sharing records found to delete.")